Considering Adding:
- Confidence intervals for performance metrics (Most papers only really seem to go this far)
- Statistical significance tests between different approaches
- Variance analysis across multiple runs




In [ ]:
!pip install optuna

In [ ]:
import sys
print(sys.executable)


import optuna
print(optuna.__version__)


### **This experiment is investigating the performance of an adaptive reward function to state of the art reward functions in environments with environmentally variable changes.**

-> Is there a statisitcally significant improvement in performance over time in this varying environment.


In [ ]:
import gymnasium as gym
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import os
import random
import torch

import sys
from pathlib import Path


current_dir = os.getcwd()  
project_root = str(Path(current_dir).parent.parent)
sys.path.append(project_root)


# Initialize environment and device
from AdaptiveRewardFunctionLearning.Prompts.prompts import device, apiKey,modelName

#Cu stomCartPoleEnv
from RLEnvironment.env import CustomCartPoleEnv
#RewardUpdateSystem
from AdaptiveRewardFunctionLearning.RewardGeneration.rewardCritic import RewardUpdateSystem
#DQLearningAgent
from RLEnvironment.training.agent import DQLearningAgent
from RLEnvironment.training.training import trainDQLearning

#DynamicRewardFunction
from AdaptiveRewardFunctionLearning.RewardGeneration.rewardCodeGeneration import stabilityReward, efficiencyReward, dynamicRewardFunction

#import
from AdaptiveRewardFunctionLearning.Visualisation.trainingTestFunctions import (
    runEpisode,
    detectJumps,
    analyzeRewardSensibility,
    performUpdate,
    updateCompositeRewardFunction,
    plotExperimentResults,
    savePlot
)

# Import new reward functions
from AdaptiveRewardFunctionLearning.RewardGeneration.cartpole_energy_reward import EnergyBasedRewardFunction


# from AdaptiveRewardFunctionLearning.RewardGeneration.cartpole_meta_learning import meta_learning_cartpole
# from AdaptiveRewardFunctionLearning.RewardGeneration.reward_meta_learning import RewardFunctionMetaLearner

**State of the Art Reward Functions with Reference to Papers**

**Potential-based Reward Shaping (PBRS):**
```python
def potentialBasedRewardShaping(observation, action):
    x, xDot, angle, angleDot = observation
    gamma = 0.99  # Example discount factor

    def phi(x, xDot, angle, angleDot):
        # Example potential function
        return -abs(x) - abs(angle)

    current_potential = phi(x, xDot, angle, angleDot)
    next_potential = phi(x + xDot, angle + angleDot, xDot, angleDot)  # Simplified next state
    return float(gamma * next_potential - current_potential)
```

Paper: "Potential-based Shaping in Model-based Reinforcement Learning"

Link: https://cdn.aaai.org/AAAI/2008/AAAI08-096.pdf


**Parameterized Reward Shaping:**
```python
def parameterizedRewardShaping(observation, action):
    x, xDot, angle, angleDot = observation
    original_reward = 1.0  # Assuming default CartPole reward

    def f(x, xDot, angle, angleDot):
        # Example shaping reward function
        return -abs(angle)

    def z_phi(x, xDot, angle, angleDot):
        # Example shaping weight function
        return 0.5

    shaping_reward = f(x, xDot, angle, angleDot)
    shaping_weight = z_phi(x, xDot, angle, angleDot)
    return float(original_reward + shaping_weight * shaping_reward)
```

Paper: "Learning to Utilize Shaping Rewards: A New Approach of Reward Shaping"

Link: http://arxiv.org/pdf/2011.02669.pdf


**Energy Based Reward Function - Physics Based**

```python
def energyBasedReward(observation, action):
    x, xDot, angle, angleDot = observation
    
    # Calculate kinetic and potential energy components
    kineticEnergy = 0.5 * (xDot**2 + angleDot**2)
    potentialEnergy = 9.8 * (1 + cos(angle))  # g * h, where h depends on angle
    
    # Reward is inverse of total energy (less energy = more stable = better reward)
    energyPenalty = -(kineticEnergy + potentialEnergy)
    return float(1.0 + 0.1 * energyPenalty)  # Base reward plus energy term
```

Paper: "Energy-Based Control for Safe Robot Learning" (2019)

Link: https://ieeexplore.ieee.org/document/8794207


**Baseline Reward Function:**
```python
def baselineCartPoleReward(observation, action):
    return 1.0
```

### **Performance Experiment**

In [ ]:

# Initialize reward functions and meta-learners
energy_reward = EnergyBasedRewardFunction(mass_cart=1.0, mass_pole=0.1, length=0.5, gravity=9.8)
# meta_reward = RewardFunctionMetaLearner(state_dim=4, action_dim=1)  # CartPole has 4 state dims, 1 action dim

# def potentialBasedRewardShaping(observation, action):
#     """Advanced potential-based reward shaping using meta-learning"""
#     reward_func = meta_reward.generate_reward_function()
#     return float(reward_func(observation, action))

# def parameterizedRewardShaping(observation, action):
#     """Meta-learning based parameterized reward shaping"""
#     # Use meta-learning framework for parameter optimization
#     learner = meta_learning_cartpole()
#     return float(learner.parameterized_reward(observation, action))

def energyBasedReward(observation, action):
    """Enhanced physics-based energy reward"""
    return float(energy_reward.compute_reward(observation, action))


def potentialBasedReward(observation, action):
    """Potential-based reward shaping for CartPole  - This one is not dynamic"""
    x, x_dot, theta, theta_dot = observation
    gamma = 0.99
    
    def potential(state):
        # Potential function based on cart position and pole angle
        # Higher potential for centered cart and upright pole
        cart_potential = -(state[0] ** 2)  # Penalize distance from center
        angle_potential = -((state[2] ** 2))  # Penalize angle from vertical
        velocity_potential = -(state[1] ** 2)  # Penalize high velocities
        ang_velocity_potential = -(state[3] ** 2)  # Penalize high angular velocities
        
        return cart_potential + 2*angle_potential + velocity_potential + ang_velocity_potential

    current_potential = potential(observation)
    next_potential = potential([x + x_dot, x_dot, theta + theta_dot, theta_dot])
    
    # PBRS formula: γΦ(s') - Φ(s)
    shaped_reward = gamma * next_potential - current_potential
    
    return 1.0 + shaped_reward


def baselineReward(observation, action):
    """Standard baseline reward"""
    return 1.0

In [ ]:
def runPerformanceComparisonTest(
    episodes=1000, 
    changeInterval=500, 
    lengthchanges=[0.5, 1.5],
    mass_cart=1.0,
    mass_pole=0.1,
    initial_length=0.5,
    gravity=9.8,
    seed=42,
    max_updates_per_component=3  # NEW parameter to limit API calls
):
    print(f"Starting Performance Comparison Test with seed {seed}...")
    
    # Set all random seeds
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    
    def create_fresh_env():
        # Helper function to create fresh environment with consistent settings
        env = gym.make('CartPole-v1', max_episode_steps=20000, render_mode=None)
        env.action_space.seed(seed)
        env.observation_space.seed(seed)
        env.reset(seed=seed)
        env = CustomCartPoleEnv(env, numComponents=2)
        env.setEnvironmentParameters(masscart=mass_cart, length=lengthchanges[0], gravity=gravity)
        return env
    
    def create_dqn_agent(env, device):
        return DQLearningAgent(
            env=env, 
            stateSize=4, 
            actionSize=2, 
            device=device,
            learningRate=0.0007,       # Balanced for adaptation and stability
            discountFactor=0.99,       # Standard value works well
            epsilon=1.0,
            epsilonDecay=0.9997,       # Slower decay preserves exploration
            epsilonMin=0.07,           # Higher minimum exploration rate
            replayBufferSize=30000,    # Smaller to adapt faster to changes
            batchSize=48,              # Moderate batch size
            targetUpdateFreq=125       # More frequent updates
        )
    
    # Initialize energy-based reward function
    energy_reward = EnergyBasedRewardFunction(
        mass_cart=mass_cart, 
        mass_pole=mass_pole, 
        length=initial_length, 
        gravity=gravity
    )
    
    # Create initial environment
    env = create_fresh_env()
    
    # Define reward function configurations
    rewardfunctions = {
        'adaptivereward': {
            'agent': None,  # Will be created fresh for each test
            'updatesystem': RewardUpdateSystem(apiKey, modelName, max_updates_per_run=max_updates_per_component),
            'rewardfunction': None,
            'update_method': 'llm'
        },
        'pbrs': {
            'agent': None,  # Will be created fresh for each test
            'updatesystem': None,
            'rewardfunction': potentialBasedReward,
            'update_method': None
        },
        'energy_based': {
            'agent': None,  # Will be created fresh for each test
            'updatesystem': energy_reward,
            'rewardfunction': energy_reward.compute_reward,
            'update_method': 'physics'
        },
        'baseline': {
            'agent': None,  # Will be created fresh for each test
            'updatesystem': None,
            'rewardfunction': baselineReward,
            'update_method': None
        }
    }

    results = {}
    test_order = ['adaptivereward', 'energy_based', 'baseline', 'pbrs']
    
    # Test each reward function
    for rewardname in test_order:
        print(f"\nTesting reward function: {rewardname}")
        
        # Create fresh environment and agent for each test
        env = create_fresh_env()
        rewardfunctions[rewardname]['agent'] = create_dqn_agent(env, device)
        rewardinfo = rewardfunctions[rewardname]
        
        # Reset length index for each test
        currentlengthidx = 0
        
        if rewardname == 'adaptivereward':
            # Initialize both components for adaptive reward
            env.setComponentReward(1, stabilityReward)
            env.setComponentReward(2, efficiencyReward)
            rewardinfo['updatesystem'].lastUpdateEpisode = 0
            print(f"Limited to {max_updates_per_component} updates per component")
        else:
            env.setRewardFunction(rewardinfo['rewardfunction'])
        
        episoderewards = []
        episodebalancetimes = []
        rewardchangeepisodes = []
        
        def onEpisodeEnd(env, updatesystem, episode, reward, steps):
            nonlocal episoderewards, episodebalancetimes, rewardchangeepisodes, currentlengthidx
            
            # Record episode results
            episoderewards.append(reward)
            episodebalancetimes.append(steps)
            
            # Create metrics dictionary
            metrics = {
                'currentEpisode': episode,
                'recentRewards': episoderewards[-100:] if len(episoderewards) > 100 else episoderewards,
                'averageBalanceTime': np.mean(episodebalancetimes[-100:]) if episodebalancetimes else 0,
                'balanceTimeVariance': np.var(episodebalancetimes[-100:]) if len(episodebalancetimes) > 1 else 0
            }
            
            # Debug print for metrics
            if episode % 1000 == 0:
                print(f"\nMetrics Debug at Episode {episode}:")
                print(f"Recent Average Reward: {np.mean(metrics['recentRewards']):.2f}")
                print(f"Average Balance Time: {metrics['averageBalanceTime']:.2f}")
                print(f"Balance Time Variance: {metrics['balanceTimeVariance']:.2f}")
                
                if rewardname == 'adaptivereward' and hasattr(env, 'getCurrentWeights'):
                    weights = env.getCurrentWeights()
                    print(f"Component Weights - Stability: {weights['stability']:.2f}, "
                          f"Efficiency: {weights['efficiency']:.2f}")
                
                # Print update statistics for adaptive reward
                if rewardname == 'adaptivereward' and hasattr(updatesystem, 'update_counts'):
                    print(f"Update counts - Component 1: {updatesystem.update_counts.get(1, 0)}/{updatesystem.max_updates_per_run}, "
                          f"Component 2: {updatesystem.update_counts.get(2, 0)}/{updatesystem.max_updates_per_run}")
            
            # Handle LLM updates only for adaptive reward
            if rewardname == 'adaptivereward' and updatesystem is not None:
                if episode % 1000 == 0:
                    print(f"\nEpisode {episode} - Time Since Last Update: {episode - updatesystem.lastUpdateEpisode}")
                
                for component in range(1, 3):
                    updatesystem.targetComponent = component
                    if updatesystem.waitingTime(f'component_{component}', metrics, updatesystem.lastUpdateEpisode):
                        current_func = env.rewardComponents[f'rewardFunction{component}']
                        new_function, updated = updatesystem.validateAndUpdate(current_func)
                        
                        if updated:
                            env.setComponentReward(component, new_function)
                            rewardchangeepisodes.append(episode)
                            updatesystem.lastUpdateEpisode = episode
                            print(f"✓ LLM update for component {component} at episode {episode}")
            
            # Handle physics-based updates
            elif rewardinfo['update_method'] == 'physics':
                if episode % changeInterval == 0 and episode > 0:
                    print(f"\nUpdating physics-based reward at episode {episode}")
                    updatesystem.length = lengthchanges[currentlengthidx]
                    env.setRewardFunction(updatesystem.compute_reward)
                    rewardchangeepisodes.append(episode)
                    print("✓ Physics-based update completed")
            
            # Environment changes
            if episode % changeInterval == 0 and episode > 0:
                currentlengthidx = (currentlengthidx + 1) % len(lengthchanges)
                newlength = lengthchanges[currentlengthidx]
                env.setEnvironmentParameters(length=newlength)
                print(f"\nChanged pole length to: {newlength}m at episode {episode}")
        
        # Train the agent
        agent, env, rewards = trainDQLearning(
            agent=rewardinfo['agent'],
            env=env,
            numEpisodes=episodes,
            updateSystem=rewardinfo['updatesystem'],
            onEpisodeEnd=onEpisodeEnd
        )
        
        # Store results
        results[rewardname] = {
            'rewards': episoderewards,
            'balancetimes': episodebalancetimes,
            'rewardChanges': rewardchangeepisodes
        }
        
        # Print final performance metrics
        print(f"\nCompleted testing {rewardname}")
        print(f"Final average reward: {np.mean(episoderewards[-100:]):.2f}")
        print(f"Final average balance time: {np.mean(episodebalancetimes[-100:]):.2f}")
        
        # Print update statistics for adaptive reward
        if rewardname == 'adaptivereward' and 'updatesystem' in rewardinfo and hasattr(rewardinfo['updatesystem'], 'update_counts'):
            print(f"Final update counts - Component 1: {rewardinfo['updatesystem'].update_counts.get(1, 0)}/{rewardinfo['updatesystem'].max_updates_per_run}, "
                  f"Component 2: {rewardinfo['updatesystem'].update_counts.get(2, 0)}/{rewardinfo['updatesystem'].max_updates_per_run}")
    
    return results

In [ ]:
# Run Experiment

changeInterval = 20000

# results = runPerformanceComparisonTest(
#     episodes=40000,  
#     changeInterval=changeInterval,
#     mass_cart=1.0,
#     lengthchanges=[0.3, 0.9]  
# )

In [ ]:
def visualizePerformanceComparison(results, changeInterval, folder_path):
    """
    Create and save performance comparison visualizations
    """
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 15))
    
    # Color map for different reward functions
    colors = ['b', 'g', 'r', 'c', 'm']
    
    # Plot rewards for each reward function with variance
    for idx, (rewardname, rewardresults) in enumerate(results.items()):
        rewards = pd.Series(rewardresults['rewards'])
        
        # Calculate rolling mean and standard deviation for rewards
        window = 50
        rolling_mean_rewards = rewards.rolling(window=window).mean()
        rolling_std_rewards = rewards.rolling(window=window).std()
        
        # Plot mean line for rewards
        ax1.plot(rolling_mean_rewards, 
                label=f'{rewardname}', 
                linewidth=2, 
                color=colors[idx])
        
        # Plot variance area for rewards
        ax1.fill_between(
            range(len(rewards)),
            rolling_mean_rewards - rolling_std_rewards,
            rolling_mean_rewards + rolling_std_rewards,
            color=colors[idx],
            alpha=0.2
        )
        
        # Add vertical lines for environment changes (red)
        change_episodes = range(changeInterval, len(rewards), changeInterval)
        for ep in change_episodes:
            ax1.axvline(x=ep, color='r', linestyle='--', alpha=0.3,
                       label='Environment Change' if ep == change_episodes[0] else None)
        
        # Add vertical lines for reward function changes (green)
        if 'rewardChanges' in rewardresults:
            for ep in rewardresults['rewardChanges']:
                ax1.axvline(x=ep, color='g', linestyle='--', alpha=0.3,
                          label=f'{rewardname} Update' if ep == rewardresults['rewardChanges'][0] else None)
    
    ax1.set_title('Average Reward Over Time with Variance')
    ax1.set_xlabel('Episode')
    ax1.set_ylabel('Reward')
    ax1.legend()
    ax1.grid(True)
    
    # Plot balance times
    for idx, (rewardname, rewardresults) in enumerate(results.items()):
        balancetimes = pd.Series(rewardresults['balancetimes'])
        
        rolling_mean_balance = balancetimes.rolling(window=window).mean()
        rolling_std_balance = balancetimes.rolling(window=window).std()
        
        ax2.plot(rolling_mean_balance,
                label=f'{rewardname}', 
                linewidth=2, 
                color=colors[idx])
        
        ax2.fill_between(
            range(len(balancetimes)),
            rolling_mean_balance - rolling_std_balance,
            rolling_mean_balance + rolling_std_balance,
            color=colors[idx],
            alpha=0.2
        )
        
        # Add vertical lines for environment changes
        for ep in change_episodes:
            ax2.axvline(x=ep, color='r', linestyle='--', alpha=0.3,
                       label='Environment Change' if ep == change_episodes[0] else None)
        
        # Add vertical lines for reward function changes
        if 'rewardChanges' in rewardresults:
            for ep in rewardresults['rewardChanges']:
                ax2.axvline(x=ep, color='g', linestyle='--', alpha=0.3,
                          label=f'{rewardname} Update' if ep == rewardresults['rewardChanges'][0] else None)
    
    ax2.set_title('Average Balance Time Over Episodes with Variance')
    ax2.set_xlabel('Episode')
    ax2.set_ylabel('Steps')
    ax2.legend()
    ax2.grid(True)
    
    # Plot environment parameters
    env_param_history = []
    for episode in range(len(next(iter(results.values()))['rewards'])):
        idx = (episode // changeInterval) % 2
        length = 0.3 if idx == 0 else 0.9  # Alternating between 0.3 and 0.9
        env_param_history.append(length)
    
    ax3.plot(env_param_history, label='Pole Length', color='purple')
    ax3.set_title('Environment Parameters Over Episodes')
    ax3.set_xlabel('Episode')
    ax3.set_ylabel('Pole Length (m)')
    ax3.grid(True)
    ax3.legend()
    
    plt.tight_layout()
    
    # Save plots
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filepath = os.path.join(folder_path, f"performance_comparison_{timestamp}.png")
    plt.savefig(filepath, bbox_inches='tight', dpi=300)
    print(f"Saved plot: performance_comparison_{timestamp}.png in {folder_path}")
    plt.close()
    
    # Create separate plots for each reward function
    visualizeSeparateRewardFunctions(results, changeInterval, folder_path)


def visualizeSeparateRewardFunctions(results, changeInterval, folder_path):
    """
    Create and save separate plots for each reward function
    """
    # Color map for different reward functions
    colors = {'adaptivereward': 'b', 'energy_based': 'g', 'baseline': 'r', 'pbrs': 'c'}
    
    # Create a 2x2 grid layout for separate plots
    fig, axs = plt.subplots(2, 2, figsize=(16, 12))
    axs = axs.flatten()
    
    # Plot each reward function in its own subplot
    for idx, (rewardname, rewardresults) in enumerate(results.items()):
        ax = axs[idx]
        
        # Get data
        rewards = pd.Series(rewardresults['rewards'])
        balancetimes = pd.Series(rewardresults['balancetimes'])
        
        # Calculate rolling mean and standard deviation
        window = 50
        rolling_mean_rewards = rewards.rolling(window=window).mean()
        rolling_std_rewards = rewards.rolling(window=window).std()
        rolling_mean_balance = balancetimes.rolling(window=window).mean()
        
        # Create twin axis for balance times
        ax2 = ax.twinx()
        
        # Plot rewards
        color = colors.get(rewardname, 'b')
        ax.plot(rolling_mean_rewards, 
                label='Reward', 
                linewidth=2, 
                color=color)
        
        # Plot variance area for rewards
        ax.fill_between(
            range(len(rewards)),
            rolling_mean_rewards - rolling_std_rewards,
            rolling_mean_rewards + rolling_std_rewards,
            color=color,
            alpha=0.2
        )
        
        # Plot balance times on second y-axis
        ax2.plot(rolling_mean_balance,
                label='Balance Time', 
                linewidth=2, 
                color='darkgreen',
                linestyle='--')
        
        # Add vertical lines for environment changes (red)
        change_episodes = range(changeInterval, len(rewards), changeInterval)
        for ep in change_episodes:
            ax.axvline(x=ep, color='r', linestyle='--', alpha=0.3,
                      label='Environment Change' if ep == change_episodes[0] else None)
        
        # Add vertical lines for reward function changes (green)
        if 'rewardChanges' in rewardresults and rewardresults['rewardChanges']:
            for ep in rewardresults['rewardChanges']:
                ax.axvline(x=ep, color='g', linestyle='--', alpha=0.3,
                          label=f'Update' if ep == rewardresults['rewardChanges'][0] else None)
        
        # Add title and labels
        ax.set_title(f'{rewardname} Performance', fontsize=14)
        ax.set_xlabel('Episode', fontsize=10)
        ax.set_ylabel('Reward', fontsize=10)
        ax2.set_ylabel('Balance Time (steps)', fontsize=10)
        
        # Add legends for both axes
        lines1, labels1 = ax.get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        ax.legend(lines1 + lines2, labels1 + labels2, loc='upper left', fontsize=8)
        
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    # Save separate plots
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filepath = os.path.join(folder_path, f"separate_reward_functions_{timestamp}.png")
    plt.savefig(filepath, bbox_inches='tight', dpi=300)
    print(f"Saved plot: separate_reward_functions_{timestamp}.png in {folder_path}")
    plt.close()
    
    
def calculateStability(rewards):
    """
    Calculate stability score based on reward variance in the last 100 episodes
    Lower variance = higher stability
    """
    if len(rewards) < 100:
        return 0.0
    
    last_hundred = rewards[-100:]
    mean_reward = np.mean(last_hundred)
    if mean_reward == 0:
        return 0.0
        
    # Calculate coefficient of variation (normalized standard deviation)
    stability = 1 - (np.std(last_hundred) / mean_reward)
    return max(0, min(1, stability))  # Normalize between 0 and 1

def calculateConvergenceTime(rewards, threshold=195, window=50):
    """
    Calculate the number of episodes needed to reach and maintain a certain performance
    threshold for a given window of episodes
    """
    if len(rewards) < window:
        return len(rewards)
    
    rolling_mean = pd.Series(rewards).rolling(window).mean()
    
    for episode in range(window, len(rewards)):
        if rolling_mean[episode] >= threshold:
            # Check if performance is maintained
            maintained = all(avg >= threshold * 0.9 for avg in rolling_mean[episode:episode+window])
            if maintained:
                return episode
    
    return len(rewards)  # If never converged, return total episodes

def calculatePerformanceMetrics(results):
    """Calculate performance metrics for each reward type"""
    metrics = {}
    for rewardname, rewardresults in results.items():
        metrics[rewardname] = {
            'finalavgreward': np.mean(rewardresults['rewards'][-100:]),
            'finalavgbalance': np.mean(rewardresults['balancetimes'][-100:]),
            'convergencetime': calculateConvergenceTime(rewardresults['rewards']),
            'stability': calculateStability(rewardresults['rewards'])
        }
    return pd.DataFrame(metrics).T
    
    
def saveMetricsTable(metrics, filename, folder_path):
    """Save metrics table to specified folder"""
    # Create figure for metrics table
    fig, ax = plt.subplots(figsize=(12, 4))
    ax.axis('tight')
    ax.axis('off')
    
    # Create table with formatted metrics
    table = ax.table(
        cellText=metrics.values.round(3),
        colLabels=metrics.columns,
        rowLabels=metrics.index,
        cellLoc='center',
        loc='center'
    )
    
    # Adjust font size and scaling
    table.auto_set_font_size(False)
    table.set_fontsize(9)
    table.scale(1.2, 1.5)
    
    plt.title("Performance Metrics Comparison")
    
    # Save with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filepath = os.path.join(folder_path, f"{filename}_{timestamp}.png")
    plt.savefig(filepath, bbox_inches='tight', dpi=300)
    print(f"Saved metrics table: {filename}_{timestamp}.png in {folder_path}")
    plt.close()

In [ ]:
def adaptation_metrics(results, changeInterval=5000):
    """
    Calculate adaptation metrics for different reward approaches:
    - Recovery time: Episodes needed to recover after an environment change
    - Performance drop: Percentage drop in reward after environment change
    - Adaptation effectiveness: How quickly and completely the agent adapts
    
    Args:
        results: Dictionary containing results from different reward approaches
        changeInterval: Number of episodes between environment changes
    
    Returns:
        DataFrame with adaptation metrics for each reward approach
    """
    adaptation_data = {}
    
    for rewardname, rewarddata in results.items():
        rewards = rewarddata['rewards']
        balancetimes = rewarddata['balancetimes']
        
        # Find the episodes where environment changes occurred
        change_episodes = list(range(changeInterval, len(rewards), changeInterval))
        
        # Skip if there aren't enough episodes to analyze
        if not change_episodes or change_episodes[0] >= len(rewards):
            continue
            
        recovery_times = []
        performance_drops = []
        adaptation_effectiveness = []
        
        for change_ep in change_episodes:
            # Skip if we don't have enough data after this change
            if change_ep + changeInterval > len(rewards):
                continue
                
            # Get data before change (baseline performance)
            pre_change_window = 100  # Last 100 episodes before change
            pre_change_start = max(0, change_ep - pre_change_window)
            pre_change_rewards = rewards[pre_change_start:change_ep]
            pre_change_avg = np.mean(pre_change_rewards) if pre_change_rewards else 0
            
            # Get data after change
            post_change_window = min(changeInterval, len(rewards) - change_ep)
            post_change_rewards = rewards[change_ep:change_ep + post_change_window]
            
            # Calculate immediate performance drop
            initial_post_avg = np.mean(post_change_rewards[:20]) if len(post_change_rewards) >= 20 else np.mean(post_change_rewards)
            performance_drop = max(0, (pre_change_avg - initial_post_avg) / pre_change_avg if pre_change_avg > 0 else 0)
            performance_drops.append(performance_drop)
            
            # Calculate recovery time (episodes to reach 90% of pre-change performance)
            recovery_threshold = 0.9 * pre_change_avg
            recovery_ep = change_ep
            for i, reward in enumerate(post_change_rewards):
                # Use sliding window average for more stable measure
                window_size = min(10, i + 1)
                recent_avg = np.mean(post_change_rewards[i - window_size + 1:i + 1])
                if recent_avg >= recovery_threshold:
                    recovery_ep = change_ep + i
                    break
            
            recovery_time = recovery_ep - change_ep
            recovery_times.append(recovery_time)
            
            # Calculate adaptation effectiveness (area under recovery curve)
            # Higher values mean faster and more complete recovery
            if recovery_time > 0 and pre_change_avg > 0:
                # Calculate area between actual performance and ideal performance
                ideal_area = pre_change_avg * recovery_time
                actual_area = sum(post_change_rewards[:recovery_time])
                effectiveness = actual_area / ideal_area if ideal_area > 0 else 0
                adaptation_effectiveness.append(effectiveness)
        
        # Store metrics
        adaptation_data[rewardname] = {
            'avg_recovery_time': np.mean(recovery_times) if recovery_times else float('nan'),
            'avg_performance_drop': np.mean(performance_drops) if performance_drops else float('nan'),
            'avg_adaptation_effectiveness': np.mean(adaptation_effectiveness) if adaptation_effectiveness else float('nan')
        }
    
    return pd.DataFrame(adaptation_data).T

def analyze_adaptation_speed(results, changeInterval=5000):
    """
    Analyze how quickly each approach adapts to environment changes,
    creating a visualization of adaptation speed with confidence intervals.
    
    Args:
        results: List of dictionaries containing multiple runs of results
        changeInterval: Number of episodes between environment changes
    
    Returns:
        Figure showing adaptation speed for each approach
    """
    # Combine data from all runs
    combined_data = {}
    
    # First, organize data by reward type and change episode
    for run_results in results:
        for rewardname, rewarddata in run_results.items():
            if rewardname not in combined_data:
                combined_data[rewardname] = {}
                
            rewards = rewarddata['rewards']
            change_episodes = list(range(changeInterval, len(rewards), changeInterval))
            
            for change_ep in change_episodes:
                if change_ep >= len(rewards):
                    continue
                    
                # Only look at episodes after change, up to next change
                post_window = min(changeInterval, len(rewards) - change_ep)
                post_rewards = rewards[change_ep:change_ep + post_window]
                
                # Get performance before change as baseline
                pre_window = 100
                pre_start = max(0, change_ep - pre_window)
                pre_rewards = rewards[pre_start:change_ep]
                pre_avg = np.mean(pre_rewards) if pre_rewards else 0
                
                # Normalize post-change rewards as percentage of pre-change performance
                normalized_rewards = []
                for r in post_rewards:
                    normalized = r / pre_avg if pre_avg > 0 else 0
                    normalized_rewards.append(normalized)
                
                # Create or update change episode data
                change_key = f"change_{change_ep}"
                if change_key not in combined_data[rewardname]:
                    combined_data[rewardname][change_key] = []
                    
                combined_data[rewardname][change_key].append(normalized_rewards)
    
    # Create visualization
    fig, ax = plt.subplots(figsize=(14, 8))
    
    # Colors for different reward approaches
    colors = {
        'adaptivereward': 'blue',
        'energy_based': 'green',
        'baseline': 'red',
        'pbrs': 'purple'
    }
    
    # Analysis window after change (how many episodes to show)
    analysis_window = 500
    
    # Plot each reward approach
    for rewardname, change_data in combined_data.items():
        # Average all change episodes for this reward approach
        all_normalized = []
        
        for change_key, runs in change_data.items():
            # First ensure all runs have same length by padding shorter ones
            max_len = min(analysis_window, max(len(run) for run in runs))
            padded_runs = []
            for run in runs:
                if len(run) < max_len:
                    padded = run + [run[-1]] * (max_len - len(run))  # Pad with last value
                else:
                    padded = run[:max_len]
                padded_runs.append(padded)
            
            # Now we can safely calculate means and CIs
            all_normalized.extend(padded_runs)
        
        # Convert to numpy array for easier calculations
        all_normalized = np.array(all_normalized)
        
        # Calculate mean and confidence intervals
        means = np.mean(all_normalized, axis=0)
        std_devs = np.std(all_normalized, axis=0)
        ci = 1.96 * std_devs / np.sqrt(all_normalized.shape[0])  # 95% CI
        
        # Plot mean line
        episodes = np.arange(len(means))
        ax.plot(episodes, means, label=rewardname, color=colors.get(rewardname, 'black'), linewidth=2)
        
        # Plot confidence interval
        ax.fill_between(episodes, means - ci, means + ci, color=colors.get(rewardname, 'black'), alpha=0.2)
    
    # Add horizontal line at 100% (pre-change performance)
    ax.axhline(y=1.0, color='gray', linestyle='--', alpha=0.7, label='Pre-change level')
    
    # Add labels and legend
    ax.set_title('Adaptation Speed After Environment Changes', fontsize=16)
    ax.set_xlabel('Episodes After Change', fontsize=14)
    ax.set_ylabel('Performance (% of pre-change level)', fontsize=14)
    ax.grid(True, alpha=0.3)
    ax.legend(fontsize=12)
    
    # Set y-axis limits with some padding
    ax.set_ylim([0, max(1.5, ax.get_ylim()[1])])
    
    return fig

# Visualize the results
# visualizePerformanceComparison(results,changeInterval)


# # Calculate and display the metrics
# metrics = calculatePerformanceMetrics(results)
# print("\nPerformance Metrics:")
# print(metrics)
    
# Call the function
# saveMetricsTable(metrics)

### Multiple runs to generate confidence intervals

In [ ]:
def createExperimentFolder():
    """Create a timestamped folder for experiment results"""
    from datetime import datetime
    import os
    
    # Create base experiment folder if it doesn't exist
    if not os.path.exists("PerformanceExperiment"):
        os.makedirs("PerformanceExperiment")
    
    # Create timestamped subfolder
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    experiment_folder = os.path.join("PerformanceExperiment", f"experiment_{timestamp}")
    os.makedirs(experiment_folder)
    
    return experiment_folder

def savePlot(fig, filename, folder_path):
    """Save plot to specified folder with timestamp"""
    from datetime import datetime
    import os
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filepath = os.path.join(folder_path, f"{filename}_{timestamp}.png")
    fig.savefig(filepath, bbox_inches='tight', dpi=300)
    print(f"Saved plot: {filename}_{timestamp}.png in {folder_path}")

def create_reward_over_time_plot_with_variance(results, changeInterval, save_path=None, include_variance=True):
    """
    Create a reward over time plot with optional variance bands.
    Shows all reward functions on one plot with clear markers for environment changes.
    
    Args:
        results: Dictionary containing results from each reward approach
        changeInterval: Number of episodes between environment changes
        save_path: Directory to save the visualization (optional)
        include_variance: Whether to include variance bands (default: True)
    
    Returns:
        Matplotlib figure object
    """
    plt.figure(figsize=(14, 8))
    
    # Define distinguishable colors and line styles
    colors = {
        'adaptivereward': '#1f77b4',  # blue
        'energy_based': '#2ca02c',    # green
        'pbrs': '#ff7f0e',            # orange
        'baseline': '#d62728'         # red
    }
    
    line_styles = {
        'adaptivereward': '-',
        'energy_based': '-.',
        'pbrs': '--',
        'baseline': ':'
    }
    
    # Plot each reward function
    for rewardname, rewardresults in results.items():
        rewards = rewardresults['rewards']
        
        # Apply smoother with larger window for cleaner visualization
        window = 50
        smoothed_rewards = pd.Series(rewards).rolling(window=window, min_periods=1).mean()
        
        if include_variance:
            # Calculate rolling standard deviation for variance bands
            rolling_std = pd.Series(rewards).rolling(window=window, min_periods=1).std()
            
            # Create confidence interval (1 standard deviation)
            upper_bound = smoothed_rewards + rolling_std
            lower_bound = smoothed_rewards - rolling_std
            
            # Fill the area between upper and lower bounds
            plt.fill_between(
                range(len(rewards)),
                lower_bound,
                upper_bound,
                alpha=0.2,
                color=colors.get(rewardname, 'black'),
                label=f"{rewardname} variance" if rewardname == list(results.keys())[0] else None
            )
        
        plt.plot(
            range(len(rewards)), 
            smoothed_rewards,
            label=rewardname,
            color=colors.get(rewardname, 'black'),
            linestyle=line_styles.get(rewardname, '-'),
            linewidth=2.5
        )
    
    # Add vertical lines for environment changes
    change_episodes = list(range(changeInterval, len(next(iter(results.values()))['rewards']), changeInterval))
    for ep in change_episodes:
        plt.axvline(x=ep, color='red', linestyle='--', alpha=0.5,
                   label='Environment Change' if ep == change_episodes[0] else None)
    
    # Add annotations for environment changes
    for i, ep in enumerate(change_episodes):
        param_value = 0.9 if i % 2 else 0.3  # Alternating pole length (customize as needed)
        plt.annotate(
            f"Length: {param_value}m",
            xy=(ep, plt.gca().get_ylim()[1] * 0.95),
            xytext=(ep + 50, plt.gca().get_ylim()[1] * 0.95),
            arrowprops=dict(facecolor='black', shrink=0.05, width=1.5, headwidth=8),
            fontsize=10,
            bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8)
        )
    
    # Add title and labels
    title_suffix = " with Variance Bands" if include_variance else ""
    plt.title(f'Reward Performance Comparison Across Different Approaches{title_suffix}', fontsize=16)
    plt.xlabel('Episode', fontsize=14)
    plt.ylabel('Average Reward (smoothed)', fontsize=14)
    plt.grid(True, alpha=0.3)
    
    # Create a custom legend with larger markers
    plt.legend(fontsize=12, loc='upper center', bbox_to_anchor=(0.5, -0.15), 
               ncol=3, frameon=True, fancybox=True, shadow=True)
    
    # Adjust margins
    plt.tight_layout()
    
    # Save the plot if a path is provided
    if save_path:
        # Create the directory if it doesn't exist
        os.makedirs(save_path, exist_ok=True)
        
        # Use a descriptive filename
        variance_suffix = "_with_variance" if include_variance else ""
        filename = f"reward_over_time{variance_suffix}.png"
        
        # Join the path correctly
        filepath = os.path.join(save_path, filename)
        
        # Save with a good margin setting
        plt.savefig(filepath, bbox_inches='tight', dpi=300)
        print(f"Saved plot to {filepath}")
    
    # Display the plot
    plt.show()
    
    return plt.gcf()  # Return the figure object

def create_aggregate_reward_plot(all_results, changeInterval, folder_path):
    """
    Create an aggregate reward plot that shows the average performance across all runs
    with confidence intervals
    """
    # First, organize data by reward type
    reward_data = {}
    max_episodes = 0
    
    # Collect data from all runs
    for run_results in all_results:
        for reward_type, results in run_results.items():
            if reward_type not in reward_data:
                reward_data[reward_type] = []
            
            # Get reward data
            rewards = pd.Series(results['rewards']).rolling(window=100, min_periods=1).mean()
            reward_data[reward_type].append(rewards)
            max_episodes = max(max_episodes, len(rewards))
    
    # Create figure
    plt.figure(figsize=(16, 10))
    
    # Define colors and line styles
    colors = {
        'adaptivereward': '#1f77b4',  # blue
        'energy_based': '#2ca02c',    # green
        'pbrs': '#ff7f0e',            # orange
        'baseline': '#d62728'         # red
    }
    
    line_styles = {
        'adaptivereward': '-',
        'energy_based': '-.',
        'pbrs': '--',
        'baseline': ':'
    }
    
    # Plot each reward type
    for reward_type, runs in reward_data.items():
        # Make sure all runs have the same length by padding shorter ones
        padded_runs = []
        for run in runs:
            if len(run) < max_episodes:
                # Pad with NaNs
                padded = run.reindex(range(max_episodes), fill_value=np.nan)
            else:
                padded = run
            padded_runs.append(padded)
        
        # Convert to numpy array for calculations
        run_data = np.array([run.values for run in padded_runs])
        
        # Calculate mean and confidence intervals
        mean_rewards = np.nanmean(run_data, axis=0)
        if run_data.shape[0] > 1:  # Only calculate std if we have multiple runs
            std_rewards = np.nanstd(run_data, axis=0)
            ci = 1.96 * std_rewards / np.sqrt(run_data.shape[0])  # 95% CI
        else:
            ci = np.zeros_like(mean_rewards)
        
        # Plot mean line
        plt.plot(
            range(len(mean_rewards)),
            mean_rewards,
            label=reward_type,
            color=colors.get(reward_type, 'black'),
            linestyle=line_styles.get(reward_type, '-'),
            linewidth=3
        )
        
        # Plot confidence interval
        plt.fill_between(
            range(len(mean_rewards)),
            mean_rewards - ci,
            mean_rewards + ci,
            color=colors.get(reward_type, 'black'),
            alpha=0.2
        )
    
    # Add vertical lines for environment changes
    change_episodes = list(range(changeInterval, max_episodes, changeInterval))
    for i, ep in enumerate(change_episodes):
        # Skip if beyond our data
        if ep >= max_episodes:
            continue
            
        plt.axvline(
            x=ep,
            color='black',
            linestyle='--',
            alpha=0.5,
            label='Environment Change' if i == 0 else None
        )
        
        # Add annotation for the environment change
        param_value = 0.9 if i % 2 else 0.3  # Alternating pole length
        y_pos = plt.gca().get_ylim()[1] * 0.95
        plt.annotate(
            f"Length: {param_value}m",
            xy=(ep, y_pos),
            xytext=(ep + max_episodes * 0.02, y_pos),
            arrowprops=dict(facecolor='black', shrink=0.05, width=1.5, headwidth=8),
            fontsize=12,
            bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8)
        )
    
    # Add title and labels
    plt.title('Aggregate Reward Performance Across All Runs (with 95% CI)', fontsize=18)
    plt.xlabel('Episode', fontsize=16)
    plt.ylabel('Average Reward (smoothed)', fontsize=16)
    plt.grid(True, alpha=0.3)
    
    # Create a custom legend
    plt.legend(
        fontsize=14,
        loc='upper center',
        bbox_to_anchor=(0.5, -0.1),
        ncol=4,
        frameon=True,
        fancybox=True,
        shadow=True
    )
    
    # Adjust margins
    plt.tight_layout()
    
    # Save the plot
    filepath = os.path.join(folder_path, f"aggregate_reward_comparison.png")
    plt.savefig(filepath, bbox_inches='tight', dpi=300)
    print(f"Saved plot: aggregate_reward_comparison.png in {folder_path}")
    
    # Show the plot
    plt.show()
    
    return plt.gcf()
    
def saveMetricsTable(metrics, filename, folder_path):
    """Save metrics table to specified folder"""
    import matplotlib.pyplot as plt
    from datetime import datetime
    import os
    
    # Style the DataFrame for visualization
    fig, ax = plt.subplots(figsize=(12, 4))
    ax.axis('tight')
    ax.axis('off')
    table = ax.table(cellText=metrics.values,
                    colLabels=metrics.columns,
                    rowLabels=metrics.index,
                    cellLoc='center',
                    loc='center')
    
    # Adjust font size and scaling
    table.auto_set_font_size(False)
    table.set_fontsize(9)
    table.scale(1.2, 1.5)
    
    plt.title("Performance Metrics Comparison")
    
    # Save with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filepath = os.path.join(folder_path, f"{filename}_{timestamp}.png")
    plt.savefig(filepath, bbox_inches='tight', dpi=300)
    print(f"Saved metrics table: {filename}_{timestamp}.png in {folder_path}")
    
    # Show the plot
    plt.show()
    plt.close()

def run_statistical_tests(results):
    """Run statistical tests to compare performance of different reward approaches"""
    # Dictionary to store test results
    test_results = {}
    
    # Prepare data for tests
    reward_data = []
    
    # Collect episode rewards for each approach
    for reward_type, reward_info in results.items():
        rewards = np.array(reward_info['rewards'])
        balance_times = np.array(reward_info['balancetimes'])
        
        # Create a DataFrame row for each episode
        for i in range(len(rewards)):
            row = {
                'reward_type': reward_type,
                'episode': i,
                'reward': rewards[i],
                'balance_time': balance_times[i]
            }
            reward_data.append(row)
    
    # Convert to DataFrame
    df = pd.DataFrame(reward_data)
    
    # Run tests for different metrics
    metrics = ['reward', 'balance_time']
    for metric in metrics:
        # Run ANOVA
        from scipy import stats
        unique_types = df['reward_type'].unique()
        
        if len(unique_types) < 2:
            test_results[metric] = {
                'anova': {'F': np.nan, 'p': np.nan},
                'pairwise': {}
            }
            continue
            
        # Extract data for each reward type
        groups = [df[df['reward_type'] == t][metric].values for t in unique_types]
        
        # Run ANOVA
        try:
            f_stat, p_value = stats.f_oneway(*groups)
            anova_result = {'F': f_stat, 'p': p_value}
        except:
            anova_result = {'F': np.nan, 'p': np.nan}
        
        # Run Tukey's HSD post-hoc test for pairwise comparisons
        try:
            from statsmodels.stats.multicomp import pairwise_tukeyhsd
            tukey = pairwise_tukeyhsd(df[metric], df['reward_type'], alpha=0.05)
            
            # Extract pairwise results
            pairwise_results = {}
            for i, group1 in enumerate(tukey.groupsunique):
                for j, group2 in enumerate(tukey.groupsunique):
                    if i < j:  # Avoid duplicates
                        key = f"{group1} vs {group2}"
                        idx = np.where((tukey.data.group1 == group1) & (tukey.data.group2 == group2))[0]
                        if len(idx) > 0:
                            reject = tukey.reject[idx[0]]
                            pairwise_results[key] = {
                                'reject_null': bool(reject),
                                'p_value': float(tukey.pvalues[idx[0]])
                            }
            
            test_results[metric] = {
                'anova': anova_result,
                'pairwise': pairwise_results
            }
        except:
            test_results[metric] = {
                'anova': anova_result,
                'pairwise': {}
            }
    
    return test_results

def runMultipleExperiments(numRuns=4, episodes=40000, changeInterval=20000, max_updates_per_component=3):
    """
    Run multiple experiments and save results in organized folders
    
    Args:
        numRuns: Number of experiment runs to perform
        episodes: Number of episodes per run
        changeInterval: Number of episodes between environment changes
        max_updates_per_component: Maximum number of API calls per reward component per run
    """
    # Create main experiment folder
    experiment_folder = createExperimentFolder()
    
    allResults = []
    allMetrics = []
    allAdaptationMetrics = []
    aggregatedMetrics = {}
    aggregatedAdaptation = {}
    
    # Create run folders
    for run in range(numRuns):
        print(f"\n=============== Starting Run {run + 1}/{numRuns} ===============")
        
        # Create folder for this run
        run_folder = os.path.join(experiment_folder, f"run_{run + 1}")
        os.makedirs(run_folder)
        
        # Run experiment
        results = runPerformanceComparisonTest(
            episodes=episodes,
            changeInterval=changeInterval,
            mass_cart=1.0,
            lengthchanges=[0.3, 0.9],
            max_updates_per_component=max_updates_per_component
        )
        
        # Calculate metrics for this run
        metrics = calculatePerformanceMetrics(results)
        adaptation_metrics_df = adaptation_metrics(results, changeInterval)
        
        # Store results
        allResults.append(results)
        allMetrics.append(metrics)
        allAdaptationMetrics.append(adaptation_metrics_df)
        
        # Store metrics by reward type
        for idx, row in metrics.iterrows():
            if idx not in aggregatedMetrics:
                aggregatedMetrics[idx] = []
            aggregatedMetrics[idx].append(row.to_dict())
        
        # Store adaptation metrics by reward type
        for idx, row in adaptation_metrics_df.iterrows():
            if idx not in aggregatedAdaptation:
                aggregatedAdaptation[idx] = []
            aggregatedAdaptation[idx].append(row.to_dict())
        
        # Visualization section
        print("\n--- Generating visualizations for Run", run + 1, "---")
        
        # 1. Performance comparison plots (original)
        print("Creating performance comparison plots...")
        visualizePerformanceComparison(results, changeInterval, run_folder)
        
        # 2. Save metrics table
        print("Creating metrics table...")
        saveMetricsTable(metrics, f"metrics_run_{run + 1}", run_folder)
        
        # 3. Create new reward plots with and without variance
        print("Creating reward over time plots...")
        # With variance bands
        reward_plot_with_variance = create_reward_over_time_plot_with_variance(
            results, 
            changeInterval, 
            save_path=run_folder,
            include_variance=True
        )
        plt.close()
        
        # Without variance bands
        reward_plot_without_variance = create_reward_over_time_plot_with_variance(
            results, 
            changeInterval, 
            save_path=run_folder,
            include_variance=False
        )
        plt.close()
    
    # Calculate confidence intervals (95%)
    confidenceIntervals = {}
    resultsTable = pd.DataFrame()
    
    for rewardType, metrics_list in aggregatedMetrics.items():
        metrics_df = pd.DataFrame(metrics_list)
        means = metrics_df.mean()
        cis = 1.96 * metrics_df.std() / np.sqrt(numRuns)
        
        # Create row for this reward type
        resultsTable.loc[rewardType, 'Average Reward'] = f"{means['finalavgreward']:.2f} ± {cis['finalavgreward']:.2f}"
        resultsTable.loc[rewardType, 'Average Balance'] = f"{means['finalavgbalance']:.2f} ± {cis['finalavgbalance']:.2f}"
        resultsTable.loc[rewardType, 'Convergence Time'] = f"{means['convergencetime']:.2f} ± {cis['convergencetime']:.2f}"
        resultsTable.loc[rewardType, 'Stability'] = f"{means['stability']:.2f} ± {cis['stability']:.2f}"
        
        # Store the raw values for possible further analysis
        confidenceIntervals[rewardType] = {
            'finalavgreward': {'mean': means['finalavgreward'], 'ci': cis['finalavgreward']},
            'finalavgbalance': {'mean': means['finalavgbalance'], 'ci': cis['finalavgbalance']},
            'convergencetime': {'mean': means['convergencetime'], 'ci': cis['convergencetime']},
            'stability': {'mean': means['stability'], 'ci': cis['stability']}
        }
    
    # Do the same for adaptation metrics
    adaptationTable = pd.DataFrame()
    adaptationIntervals = {}
    
    for rewardType, metrics_list in aggregatedAdaptation.items():
        if not metrics_list:
            continue
            
        metrics_df = pd.DataFrame(metrics_list)
        means = metrics_df.mean()
        cis = 1.96 * metrics_df.std() / np.sqrt(numRuns)
        
        # Create row for this reward type
        for metric in metrics_df.columns:
            if means[metric] == means[metric]:  # Check for NaN
                adaptationTable.loc[rewardType, metric] = f"{means[metric]:.2f} ± {cis[metric]:.2f}"
            else:
                adaptationTable.loc[rewardType, metric] = "N/A"
        
        # Store raw values
        adaptationIntervals[rewardType] = {
            col: {'mean': means[col], 'ci': cis[col]} 
            for col in metrics_df.columns if means[col] == means[col]
        }
    
    print("\n--- Creating Aggregate Analysis ---")
    
    # Run statistical tests across all runs
    if len(allResults) > 1:
        print("Running statistical tests...")
        # Combine results from all runs
        combined_results = {}
        for reward_type in allResults[0].keys():
            combined_results[reward_type] = {
                'rewards': [],
                'balancetimes': [],
                'rewardChanges': []
            }
            
            for run_result in allResults:
                if reward_type in run_result:
                    combined_results[reward_type]['rewards'].extend(run_result[reward_type]['rewards'])
                    combined_results[reward_type]['balancetimes'].extend(run_result[reward_type]['balancetimes'])
                    if 'rewardChanges' in run_result[reward_type]:
                        combined_results[reward_type]['rewardChanges'].extend(run_result[reward_type]['rewardChanges'])
        
        # Run statistical tests
        stats_results = run_statistical_tests(combined_results)
        
        # Save statistical test results
        with open(os.path.join(experiment_folder, "statistical_tests.txt"), "w") as f:
            f.write("Statistical Test Results:\n")
            for metric, tests in stats_results.items():
                f.write(f"\n{metric.upper()} TESTS:\n")
                f.write(f"ANOVA: F={tests['anova']['F']:.4f}, p={tests['anova']['p']:.4f}\n")
                f.write("Pairwise comparisons:\n")
                for pair, result in tests['pairwise'].items():
                    significant = "Significant" if result['reject_null'] else "Not significant"
                    f.write(f"  {pair}: {significant} (p={result['p_value']:.4f})\n")
    
    # Generate and save adaptation speed visualization
    if numRuns > 1:
        print("Creating adaptation speed visualization...")
        adaptation_fig = analyze_adaptation_speed(allResults, changeInterval)
        savePlot(adaptation_fig, "adaptation_speed", experiment_folder)
        
        # Create aggregate reward plot
        print("Creating aggregate reward plot...")
        aggregate_fig = create_aggregate_reward_plot(allResults, changeInterval, experiment_folder)
        plt.close(aggregate_fig)
    
    # Save aggregate statistics
    print("Saving aggregate statistics...")
    with open(os.path.join(experiment_folder, "aggregate_statistics.txt"), "w") as f:
        f.write(f"Experiment Configuration:\n")
        f.write(f"- Number of runs: {numRuns}\n")
        f.write(f"- Episodes per run: {episodes}\n")
        f.write(f"- Environment change interval: {changeInterval}\n")
        f.write(f"- Max updates per component per run: {max_updates_per_component}\n\n")
        
        f.write("Aggregate Performance Statistics:\n")
        for rewardType, metrics in confidenceIntervals.items():
            f.write(f"\n{rewardType}:\n")
            for metric, values in metrics.items():
                f.write(f"{metric}: {values['mean']:.2f} ± {values['ci']:.2f}\n")
        
        f.write("\n\nAggregate Adaptation Statistics:\n")
        for rewardType, metrics in adaptationIntervals.items():
            f.write(f"\n{rewardType}:\n")
            for metric, values in metrics.items():
                if values['mean'] == values['mean']:  # Check if not NaN
                    f.write(f"{metric}: {values['mean']:.2f} ± {values['ci']:.2f}\n")
                else:
                    f.write(f"{metric}: N/A\n")
    
    # Save final results tables
    print("Creating final metrics tables...")
    saveMetricsTable(resultsTable, "final_results", experiment_folder)
    if not adaptationTable.empty:
        saveMetricsTable(adaptationTable, "adaptation_metrics", experiment_folder)
    
    print(f"\nExperiment results saved in: {experiment_folder}")
    return confidenceIntervals, adaptationIntervals, allResults, resultsTable, adaptationTable

In [ ]:
# Run a small test experiment with multiple runs

# This will run a smaller experiment to demonstrate the new plots
confidenceIntervals, adaptationIntervals, allResults, resultsTable, adaptationTable = runMultipleExperiments(
    numRuns=2,  # Run just 2 experiments to see the results faster
    episodes=1000,  # Shorter episodes for testing
    changeInterval=500,  # Environment changes every 500 episodes
    max_updates_per_component=3  # Limit API calls per component
)

# Display the results
print("\nFinal Performance Metrics (with confidence intervals):")
print(resultsTable)

print("\nAdaptation Metrics (with confidence intervals):")
print(adaptationTable)

# For a full experiment, use parameters like:
# confidenceIntervals, adaptationIntervals, allResults, resultsTable, adaptationTable = runMultipleExperiments(
#     numRuns=4,  # Run 4 complete experiments for statistical significance 
#     episodes=40000,  # Each experiment runs for 40,000 episodes
#     changeInterval=10000,  # Environment changes every 10,000 episodes
#     max_updates_per_component=3  # Limit to 3 updates per component per run
# )